In [27]:
import numpy as np

import tb_hamiltonian.continuum.new as TBHam

model = TBHam.BLGContinuumModel()

In [28]:
def decompose_and_keep_remainder(P, b1, b2):
    # Create a matrix B using b1 and b2 as columns
    B = np.column_stack((b1, b2))

    # Solve for m1 and m2 (not necessarily integers)
    coeffs = np.linalg.solve(B, P)

    # Round m1 and m2 to the nearest integers
    m1 = int(round(coeffs[0]))
    m2 = int(round(coeffs[1]))

    # Compute the remainder by subtracting the integer combination of b1 and b2 from P
    rest = P - (m1 * b1 + m2 * b2)

    # Return only the remainder
    return rest

In [29]:
L = 500
t1 = 2.7
tp = 0.4
V0 = 0
VSL = 0.005
alpha = 0.3

b1 = model.b1G / np.floor(2 * L / 3 / model.bond_length)
b2 = model.b2G / np.floor(2 * L / 3 / model.bond_length)

KG = (model.b1G - model.b2G) / 3

Qn = TBHam.Qn(b1)
Q_vectors = TBHam.Q_concentric(b1, b2, model.b1G, model.b2G, max_val=4)


def H2D_KG(k):
    return model.H_folded(k + KG, t1, tp, V0, VSL, alpha, Q_vectors, Qn)


K_new = decompose_and_keep_remainder(KG, b1, b2)

high_sym_points_K = {
    "Γ": np.array([0.0, 0.0]) - K_new,
    "M": b1 / 2 - K_new,
    "K": (b1 - b2) / 3 - K_new,
}

path = "Γ K M Γ"
path_points_K = [high_sym_points_K[k] for k in path.split()]
kpath_K, k_point_indices = TBHam.interpolate_path(path_points_K, total_points=200)

In [30]:
eigenvalues_K, _ = TBHam.compute_eigenstuff(H2D_KG, kpath_K)

In [31]:
KpG = -KG  # Define K'G as the negative of KG

def H2D_KpG(k):
    return model.H_folded(k + KpG, t1, tp, V0, VSL, alpha, Q_vectors, Qn)

Kp_new = -K_new

high_sym_points_Kp = {
    "Γ": np.array([0.0, 0.0]) - Kp_new,
    "M": b1 / 2 - Kp_new,
    "K": (b1 - b2) / 3 - Kp_new,
}

# Define paths and interpolate
path_points_Kp = [high_sym_points_Kp[k] for k in path.split()]

kpath_Kp, _ = TBHam.interpolate_path(path_points_Kp, total_points=200)


In [32]:
eigenvalues_Kp, _ = TBHam.compute_eigenstuff(H2D_KpG, kpath_Kp)


In [33]:
# Combine and sort eigenvalues
eigenvalues_combined = np.hstack((eigenvalues_K, eigenvalues_Kp))
eigenvalues_sorted = np.sort(eigenvalues_combined, axis=1)

In [34]:
nbands = eigenvalues_K.shape[1]
mid_band = int(nbands / 2)
EF = (np.min(eigenvalues_K[:, mid_band]) + np.max(eigenvalues_K[:, mid_band - 1])) / 2

In [35]:
from tb_hamiltonian.utils import BandStructure

bs = BandStructure(
    high_sym_points=high_sym_points_K,
    path=path,
    distances=[*range(len(eigenvalues_K))],
    eigenvalues=eigenvalues_sorted,
    high_sym_indices=k_point_indices,
    e_fermi=EF,
)

In [ ]:
bs.plot(
    fig_params={
        "figsize": (6, 6),
        "ylim": (-0.02, 0.02),
    },
    plot_params={"color": "black"},
)